In [ ]:
import os
import cv2
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

In [ ]:
SIZE = 256

BENIGN_IMAGES_DIR = 'datasets/Dataset_BUSI_with_GT/benign/'
MALIGNANT_IMAGES_DIR = 'datasets/Dataset_BUSI_with_GT/malignant/'
NORMAL_IMAGES_DIR = 'datasets/Dataset_BUSI_with_GT/normal/'

In [ ]:
def load_data(images_dir, label):
    images = []
    labels = []
    IMG_WIDTH, IMG_HEIGHT = 256, 256  # Set your desired image dimensions

    for filename in os.listdir(images_dir):
        if filename.endswith('.png') and '_mask' not in filename:
            # Load and preprocess image
            img = cv2.imread(os.path.join(images_dir, filename))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (SIZE, SIZE))
            img = img / 255.0  # Normalize image to range [0, 1]
            images.append(img)
            labels.append(label)  # Append the label for each image

    images = np.array(images)
    labels = np.array(labels)

    return images, labels

In [ ]:
normal_images, normal_labels = load_data(NORMAL_IMAGES_DIR, 0)
benign_images, benign_labels = load_data(BENIGN_IMAGES_DIR, 1)
malignant_images, malignant_labels = load_data(MALIGNANT_IMAGES_DIR, 2)

In [ ]:
# Concatenate the images and labels
all_images = np.concatenate((normal_images, benign_images, malignant_images), axis=0)
all_labels = np.concatenate((normal_labels, benign_labels, malignant_labels), axis=0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(all_images, all_labels, test_size=0.2, random_state=42)

In [ ]:
def create_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(3, activation='softmax'))  # 3 classes: normal, benign, malignant

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Create the model
input_shape = (SIZE, SIZE, 1)
cnn_model = create_cnn_model(input_shape)

# Train the model
cnn_model.fit(X_train, y_train, epochs=15, batch_size=16, validation_split=0.2)

In [ ]:
# Make predictions on the test set
y_pred = cnn_model.predict(X_test)

# Convert predictions to class labels
y_pred_classes = np.argmax(y_pred, axis=1)

# Print the first 10 predictions and their corresponding true labels
print("Predicted classes: ", y_pred_classes[:10])
print("True classes: ", y_test[:10])

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'Accuracy on the test set: {accuracy * 100:.2f}%')